#### Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import warnings
import sys
import pandas as pd

sys.path.append("../")
from pipeline import data
from pipeline.config import CONF
from pipeline.data import plots
from pipeline.data import io
from pipeline.data import inspection
from pipeline.data import preprocess

# To suppress all warnings
warnings.filterwarnings("ignore")

# black is a code formatter (see https://github.com/psf/black).
# It will automatically format the code you write in the cells imposing consistent Python style.
%load_ext jupyter_black
# matplotlib style file
# Template for style file: https://matplotlib.org/stable/tutorials/introductory/customizing.html#customizing-with-style-sheets
plt.style.use("../matplotlib_style.txt")
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

### Load raw data

In [2]:
# Load raw data
(
    Installed_Capacity_Germany,
    Prices_Europe,
    Realised_Supply_Germany,
    Realised_Demand_Germany,
    Weather_Data_Germany,
    Weather_Data_Germany_2022,
) = data.load_data(CONF=CONF, data_type="raw")

Loaded Installed_Capacity_Germany from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Installed_Capacity_Germany.csv' successfully.
Loaded Prices_Europe from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Prices_Europe.csv' successfully.
Loaded Realised_Supply_Germany from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Realised_Supply_Germany.csv' successfully.
Loaded Realised_Demand_Germany from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Reaslised_Demand_Germany.csv' successfully.
Loaded Weather_Data_Germany from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Weather_Data_Germany.csv' successfully.
Loaded Weather_Data_Germany_2022 from '/graphics/scratch2/students/nguyenlo/seminar-ml/raw_data/Weather_Data_Germany_2022.csv' successfully.


### Inspect raw data

In [ ]:
# Generate profile reports
if CONF.data.inspect:
    data.save_data_inspection(
        Installed_Capacity_Germany=Installed_Capacity_Germany,
        Prices_Europe=Prices_Europe,
        Realised_Supply_Germany=Realised_Supply_Germany,
        Realised_Demand_Germany=Realised_Demand_Germany,
        Weather_Data_Germany=Weather_Data_Germany,
        Weather_Data_Germany_2022=Weather_Data_Germany_2022,
        CONF=CONF,
        data_type="raw",
    )

### Raw data processing

##### Merging data together

In [3]:
# Merging Weather_Data_Germany and Weather_Data_Germany_2022
# Replace Weather_Data_Germany with Weather_Data_Germany_2022 for 2022 data

# Remove the data for 2022 from the original dataframe
Weather_Data_Germany = Weather_Data_Germany[
    Weather_Data_Germany["time"].dt.year != 2022
]

# Concatenate the filtered original dataframe with the 2022 data
Weather_Data_Germany = pd.concat(
    [Weather_Data_Germany, Weather_Data_Germany_2022], ignore_index=True
)

##### Fill NaN

In [4]:
Installed_Capacity_Germany = data.process_na_values(Installed_Capacity_Germany, CONF)
Prices_Europe = data.process_na_values(Prices_Europe, CONF)
Realised_Supply_Germany = data.process_na_values(Realised_Supply_Germany, CONF)
Realised_Demand_Germany = data.process_na_values(Realised_Demand_Germany, CONF)
Weather_Data_Germany = data.process_na_values(Weather_Data_Germany, CONF)

#### Split train, val, test

In [5]:
Installed_Capacity_Germany = preprocess.split_data(
    df=Installed_Capacity_Germany, column_name=io.DATE_COLUMNS[-1]
)
Prices_Europe = preprocess.split_data(df=Prices_Europe, column_name=io.DATE_COLUMNS[-1])
Realised_Supply_Germany = preprocess.split_data(
    df=Realised_Supply_Germany, column_name=io.DATE_COLUMNS[-1]
)
Realised_Demand_Germany = preprocess.split_data(
    df=Realised_Demand_Germany, column_name=io.DATE_COLUMNS[-1]
)
Weather_Data_Germany = preprocess.split_data(
    df=Weather_Data_Germany, column_name=io.DATE_COLUMNS_WEATHER[0]
)

#### Normalize data

In [7]:
Installed_Capacity_Germany = preprocess.normalize_data(
    df=Installed_Capacity_Germany, ignore_features=io.DATE_COLUMNS
)
Prices_Europe = preprocess.normalize_data(
    df=Prices_Europe, ignore_features=io.DATE_COLUMNS
)
Realised_Supply_Germany = preprocess.normalize_data(
    df=Realised_Supply_Germany, ignore_features=io.DATE_COLUMNS
)
Realised_Demand_Germany = preprocess.normalize_data(
    df=Realised_Demand_Germany, ignore_features=io.DATE_COLUMNS
)
Weather_Data_Germany = preprocess.normalize_data(
    df=Weather_Data_Germany,
    ignore_features=io.DATE_COLUMNS_WEATHER + ["longitude", "latitude"],
)

#### Aggregate weather data

In [8]:
Weather_Data_Germany.head()

,longitude,latitude,forecast_origin,time,cdir,z,msl,blh,tcc,u10,v10,t2m,ssr,tsr,sund,tp,fsr,u100,v100,train,val,test
0,5.8,54.2,2019-01-01,2019-01-01,-0.895983,-0.843734,0.861054,0.376895,-0.856847,3.477066,0.799417,-0.168077,-0.819433,-0.885197,-0.899182,-0.404491,-1.983884,2.845296,0.482135,True,False,False
1,5.8,53.2,2019-01-01,2019-01-01,-0.895983,-0.844585,1.218951,0.528094,0.821789,1.802705,0.402100,-0.268699,-0.819433,-0.885197,-0.899182,-0.404491,-1.304693,1.852911,0.301878,True,False,False
2,5.8,52.2,2019-01-01,2019-01-01,-0.895983,-0.723015,1.503220,0.567414,0.962627,1.268606,0.452042,-0.374367,-0.819433,-0.885197,-0.899182,-0.404491,-0.810872,1.366301,0.346229,True,False,False
3,5.8,51.2,2019-01-01,2019-01-01,-0.895983,-0.783375,1.737973,0.088918,0.963905,0.891587,0.189490,-0.403454,-0.819433,-0.885197,-0.899182,-0.404491,-0.260718,1.038873,0.113191,True,False,False
4,5.8,50.2,2019-01-01,2019-01-01,-0.895983,0.331537,1.885205,-0.371341,0.963905,0.852461,-0.421706,-0.711545,-0.819433,-0.885197,-0.899182,-0.404491,-1.021021,0.875331,-0.488663,True,False,False


#### Save data

In [ ]:
data.save_data(
    Installed_Capacity_Germany=Installed_Capacity_Germany,
    Prices_Europe=Prices_Europe,
    Realised_Supply_Germany=Realised_Supply_Germany,
    Realised_Demand_Germany=Realised_Demand_Germany,
    Weather_Data_Germany=Weather_Data_Germany,
    CONF=CONF,
    data_type="preprocessed",
)

### Processed data inspection

In [ ]:
# Load processed data
(
    Installed_Capacity_Germany,
    Prices_Europe,
    Realised_Supply_Germany,
    Realised_Demand_Germany,
    Weather_Data_Germany,
) = data.load_data(CONF, data_type="preprocessed")

In [ ]:
# Generate profile reports

if CONF.data.inspect:
    data.save_data_inspection(
        Installed_Capacity_Germany=Installed_Capacity_Germany,
        Prices_Europe=Prices_Europe,
        Realised_Supply_Germany=Realised_Supply_Germany,
        Realised_Demand_Germany=Realised_Demand_Germany,
        Weather_Data_Germany=Weather_Data_Germany,
        CONF=CONF,
        data_type="preprocessed",
    )

### Plots processed data

In [ ]:
plots.plot_df(Installed_Capacity_Germany, "Installed_Capacity_Germany", CONF)
plots.plot_df(Prices_Europe, "Prices_Europe", CONF)
plots.plot_df(Realised_Supply_Germany, "Realised_Supply_Germany", CONF)
plots.plot_df(Realised_Demand_Germany, "Realised_Demand_Germany", CONF)

### Exploring data


##### How many places do we have weather data to?

In [ ]:
unique_locations = Weather_Data_Germany[["latitude", "longitude"]].drop_duplicates()
number_of_unique_places = len(unique_locations)
print(f"There are {number_of_unique_places} unique places in the dataset.")

##### Date resolution

In [ ]:
inspection.date_range_and_resolution(Installed_Capacity_Germany, io.DATE_COLUMNS)

In [ ]:
inspection.date_range_and_resolution(Prices_Europe, io.DATE_COLUMNS)

In [ ]:
inspection.date_range_and_resolution(Realised_Supply_Germany, io.DATE_COLUMNS)

In [ ]:
inspection.date_range_and_resolution(Realised_Demand_Germany, io.DATE_COLUMNS)

In [ ]:
inspection.date_range_and_resolution(Weather_Data_Germany, io.DATE_COLUMNS_WEATHER)